In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [15]:
pretrained_vectors = model.wv

/home/ubuntu/.virtualenvs/hana/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [6]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

import time
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
df=pd.read_hdf('df.h5', 'table')
weight_df=pd.read_hdf('tfidf_weights_df.h5', 'table')

In [24]:
desc=df['comb_desc'].apply(lambda row: row.split(' '))

In [12]:
tfidf_dict = dict(zip(weight_df.term, weight_df.weight))

In [13]:
def tfidf_mapping(desc, word_vectors, dims):
    desc_vec=[]
    for s in desc:
        try:
            desc_vec.append(tfidf_dict[s]*word_vectors[s])
        except:
            pass
    if len(desc_vec)==0:
        return np.zeros(shape=(dims), dtype='float32')
    else:
        return np.array(desc_vec).mean(axis=0)

In [17]:
pretrained_vectors.most_similar('skirt')

2018-06-08 11:42:32,615 : INFO : precomputing L2-norms of word weight vectors


[('skirts', 0.7735446095466614),
 ('miniskirt', 0.6354944109916687),
 ('dress', 0.6281509399414062),
 ('mini_skirt', 0.6121660470962524),
 ('dresses', 0.6072989106178284),
 ('hem', 0.6056695580482483),
 ('peasant_skirt', 0.6029708385467529),
 ('pleated_skirt', 0.5962336659431458),
 ('backless_dress', 0.5954369902610779),
 ('blouse', 0.594695508480072)]

In [25]:
%%time
desc_embedding=desc.apply(lambda row: tfidf_mapping(row, pretrained_vectors,300))

CPU times: user 2min 27s, sys: 139 ms, total: 2min 27s
Wall time: 2min 27s


In [26]:
desc_embedding.head()

0    [-0.0004091055, 0.0009074633, -0.0008780065, 0...
1    [-0.00060559757, -0.00022781582, -0.0020588825...
2    [-0.00049811177, -0.00027617105, -0.0007287184...
3    [-8.5288426e-05, 0.0009226902, -4.968929e-05, ...
4    [2.4033781e-05, 0.00097301934, -0.0002808425, ...
Name: comb_desc, dtype: object

In [28]:
df['pre_trained_embedding']=desc_embedding

In [29]:
df.to_hdf('model/googlenews/df.h5','table')

/home/ubuntu/.virtualenvs/hana/lib/python3.6/site-packages/pandas/core/generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['product_id', 'designer_original_name', 'product_name', 'comb_desc', 'From', 'style', 'pre_trained_embedding']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
